Phase 1: Influence & Alignment
Focuses on whether agents explicitly reference the peers they vote for or the final winner in their justifications.
✅ Lightweight and fully supported using textual inspection and LLM tagging.



The influence analysis reveals clear variation across models in how often they explicitly reference their voting targets or the eventual winner. Models like phi4, llama3, and phi4-mini-reasoning show a higher tendency to mention the agents they vote for, suggesting intentional signaling or persuasive alignment with their choices. In contrast, models like gemma3 and qwen3 seldom referenced their vote targets, indicating either implicit decision strategies or limited rhetorical coherence. When it comes to acknowledging the final winner in their justifications, models such as llama2 and phi4-mini-reasoning stood out, while others like granite3.3 and phi4-reasoning almost never referenced the eventual victor. These patterns highlight emerging agent-level strategies in social justification, with some models appearing more explicitly relational in their voting rationale. This suggests that even in the absence of predefined roles, large language models vary in their tendency to publicly align, endorse, or anticipate influential peers in a deliberative setting.

In [22]:
import json
import pandas as pd
import os

# === CONFIGURATION ===
DATA_DIR = "./nomiclaw/nomiclaw_homogeneous"  # Update with your actual path
FILE_NAMES = [f for f in os.listdir(DATA_DIR) if f.endswith(".json")]

# === STEP 1: Parse all homogeneous files ===
records = []

for fname in FILE_NAMES:
    model_name = fname.split("_")[2]  # Extract model name from file name
    path = os.path.join(DATA_DIR, fname)

    with open(path, "r") as f:
        rounds = json.load(f)
        for round_data in rounds:
            vignette = round_data["vignette"]
            round_number = round_data["round_number"]
            winner_id = round_data.get("winner_id", None)
            vote_counts = pd.Series([vote["to"] for vote in round_data["voting_network"]]).value_counts()

            for agent_id, agent_data in round_data["agents"].items():
                vote_target = agent_data.get("vote", "")
                voting_reasoning = agent_data.get("voting_reasoning", "")
                parse_fail = "Used raw text due to parse failure" in voting_reasoning

                records.append({
                    "vignette": vignette,
                    "round": round_number,
                    "model": model_name,
                    "agent_id": agent_id,
                    "vote_target": vote_target,
                    "voting_reasoning": voting_reasoning,
                    "winner_id": winner_id,
                    "mentions_vote_target": vote_target in voting_reasoning if isinstance(vote_target, str) else False,
                    "mentions_winner": winner_id in voting_reasoning if isinstance(winner_id, str) else False,
                    "parse_fail_vote": parse_fail
                })

# === STEP 2: Build DataFrame and clean ===
df = pd.DataFrame(records)
df = df[~df["parse_fail_vote"]].reset_index(drop=True)

# === STEP 3: Compute influence alignment per model ===
grouped = df.groupby("model").agg(
    total_votes=("voting_reasoning", "count"),
    mentioned_vote_target=("mentions_vote_target", "sum"),
    mentioned_winner=("mentions_winner", "sum")
).reset_index()

grouped["Explicitly Supported Peer (%)"] = (grouped["mentioned_vote_target"] / grouped["total_votes"] * 100).round(1)
grouped["Acknowledged Final Winner (%)"] = (grouped["mentioned_winner"] / grouped["total_votes"] * 100).round(1)

# === STEP 4: Final output table ===
mention_table = grouped[[
    "model", "Explicitly Supported Peer (%)", "Acknowledged Final Winner (%)"
]].sort_values(by="Explicitly Supported Peer (%)", ascending=False)

# === STEP 5: Display or export ===
print("✅ Influence alignment metrics (homogeneous setup):\n")
print(mention_table.to_string(index=False))

# Optional: save to file
# mention_table.to_csv("influence_mentions_table_homogeneous.csv", index=False)


✅ Influence alignment metrics (homogeneous setup):

              model  Explicitly Supported Peer (%)  Acknowledged Final Winner (%)
             llama3                           76.0                           27.0
     phi4-reasoning                           48.0                           26.0
         granite3.3                           30.0                           20.0
        deepseek-r1                           29.0                           16.0
              qwen3                           16.0                            4.0
               phi4                           14.0                            8.0
             llama2                           13.0                            1.0
             gemma2                            6.0                            6.0
             gemma3                            1.0                            1.0
phi4-mini-reasoning                            1.0                            0.0


In [1]:
import json
import pandas as pd
import os

# === CONFIGURATION ===
DATA_DIR = "./nomiclaw/nomiclaw_heterogeneous"
FILE_NAMES = [f for f in os.listdir(DATA_DIR) if f.endswith(".json")]

# === STEP 1: Parse all JSON files and extract relevant data ===
records = []
for fname in FILE_NAMES:
    path = os.path.join(DATA_DIR, fname)
    with open(path, "r") as f:
        rounds = json.load(f)
        for round_data in rounds:
            vignette = round_data["vignette"]
            round_number = round_data["round_number"]
            winner_id = round_data.get("winner_id", None)
            vote_counts = pd.Series([vote["to"] for vote in round_data["voting_network"]]).value_counts()

            for agent_id, agent_data in round_data["agents"].items():
                model = agent_data.get("model", agent_id)
                votes_received = vote_counts.get(agent_id, 0)

                records.append({
                    "vignette": vignette,
                    "round": round_number,
                    "agent_id": agent_id,
                    "model": model,
                    "winner_id": winner_id,
                    "proposed_rule": agent_data.get("proposed_rule", ""),
                    "proposal_reasoning": agent_data.get("proposal_reasoning", ""),
                    "vote_target": agent_data.get("vote", None),
                    "voting_reasoning": agent_data.get("voting_reasoning", ""),
                    "cumulative_score": agent_data.get("cumulative_score", None),
                    "score_change": agent_data.get("score_change", None),
                    "votes_received": votes_received,
                    "mentions_others_proposal": any(f"agent_{i}" in agent_data.get("proposal_reasoning", "") for i in range(1, 11)),
                    "mentions_others_vote": any(f"agent_{i}" in agent_data.get("voting_reasoning", "") for i in range(1, 11)),
                    "mentions_vote_target": agent_data.get("vote", "") in agent_data.get("voting_reasoning", ""),
                    "parse_fail_proposal": "Used raw text due to parse failure" in agent_data.get("proposal_reasoning", ""),
                    "parse_fail_vote": "Used raw text due to parse failure" in agent_data.get("voting_reasoning", "")
                })

# === STEP 2: Create DataFrame ===
df = pd.DataFrame(records)

# === STEP 3: Clean rows — drop parse failures ===
clean_df = df[
    ~(df["parse_fail_proposal"] | df["parse_fail_vote"])
].reset_index(drop=True)

# === STEP 4: Save to CSV ===
out_path = "cleaned_agent_qualitative_data_heterogeneous.csv"
clean_df.to_csv(out_path, index=False)
print(f"✅ Cleaned data with scores and votes saved to {out_path} ({len(clean_df)} rows).")

import pandas as pd

# Load your cleaned heterogeneous qualitative data
df = pd.read_csv("cleaned_agent_qualitative_data_heterogeneous.csv")

# Create helper column for whether winner was mentioned in vote reasoning
df["mentions_winner"] = df.apply(
    lambda row: isinstance(row["winner_id"], str) and row["winner_id"] in str(row["voting_reasoning"]),
    axis=1
)

# Group by agent model and compute counts
grouped = df.groupby("model").agg(
    total_votes=("voting_reasoning", "count"),
    mentioned_vote_target=("mentions_vote_target", "sum"),
    mentioned_winner=("mentions_winner", "sum")
).reset_index()

# Compute percentages
grouped["Explicitly Supported Peer (%)"] = (grouped["mentioned_vote_target"] / grouped["total_votes"] * 100).round(1)
grouped["Acknowledged Final Winner (%)"] = (grouped["mentioned_winner"] / grouped["total_votes"] * 100).round(1)

# Map agent_id to model name (for clarity if needed)
model_map = {
    "agent_1": "phi4",
    "agent_2": "gemma3",
    "agent_3": "llama3",
    "agent_4": "qwen3",
    "agent_5": "phi4-reasoning",
    "agent_6": "phi4-mini-reasoning",
    "agent_7": "granite3.3",
    "agent_8": "gemma2",
    "agent_9": "deepseek-r1",
    "agent_10": "llama2"
}
grouped["model"] = grouped["model"].map(model_map)

# Final table with only needed columns
mention_table = grouped[[
    "model", "Explicitly Supported Peer (%)", "Acknowledged Final Winner (%)"
]].sort_values(by="Explicitly Supported Peer (%)", ascending=False)

# Display or save
print(mention_table.to_string(index=False))
# mention_table.to_csv("influence_mentions_table.csv", index=False)



✅ Cleaned data with scores and votes saved to cleaned_agent_qualitative_data_heterogeneous_N5.csv (784 rows).
              model  Explicitly Supported Peer (%)  Acknowledged Final Winner (%)
             llama3                           57.3                            8.0
     phi4-reasoning                           51.6                           41.1
               phi4                           34.8                            8.7
phi4-mini-reasoning                           31.4                           15.7
         granite3.3                           27.3                           14.1
        deepseek-r1                           19.8                           11.1
             gemma2                           17.0                            8.0
             llama2                           14.9                           10.6
             gemma3                            9.3                            6.2
              qwen3                            7.5                    